# Export image function notebook within ArcGIS Pro


## Notebook for Exporting Jpeg Images from NICFI, Sentinel Datasource, Esri Source.

### V2:
- Export the Esri source `World Imagery` 
- Based on the certain shapefile
- Scall setting: 
  - `< 4ha` fixed setting 5ha
  - `> 4ha` dynamic setting 2times of the shape
- Label infor in the jpeg: index, image date( from the basedmap `World Imagery` ), 
- Boundary line of the shape on the jpeg
- Only export the last imagery of the Esri world imagery


### V1:
- Target shapefile: 1823_ADRSM
- Important fields: `DEN_BOT`, `AREA_HA`, `Index`, `SHAPE@`
- Export only if `AREA_HA` > 10
- NICFI datasource imported via GEE in Python notebook
- Sentinel-2 data could be imported through several channels; this solution utilizes ArcGIS Pro Living Atlas (AWS host)
- Dynamic scale properties:
  - If size < 10ha, image will scale at 8 times 
  - If 10ha < size < 50ha, image will scale at 5 times
  - If 50ha < shape size < 100ha, image will scale at 2 times
  - If shape size > 100ha, image will scale at 1.3 times

- Five Years and 60 images for each dumpsite using NICFI  
1. Each block contains the function description
2. NICFI:
   - Get NICFI
   - Add NICFI
   - Determine the date range (check the new update)
   - Get Project, layout, map, mapview, mapframe
   - Processing other Exporting 
3. Sentinel: 
   - Add sentinel data from Living Atlas (or other resource option)
4. Export multiple date's images (5 years x 12 months) of each shape

### Environment:
- ArcGIS Pro 3.2.2
- Python 3.8.19


## Basic Function
Run basic before export.

### Parameter setting

In [45]:
# Shapefile layer name in arcgis
shapefile_layer_name = "c1823_ADRSM_01ha"

# export jpeg path and folder
export_path = r"C:\Users\lycaz\Desktop\world_imagery"

# export jpeg resolution
export_resolution = 300

# minimum  area size of the shapefile 
min_area = 0.1

# lib import 
import arcpy
import datetime
# check the shapefile return selected count of feature class
print(f"Count of shape of {shapefile_layer_name} : {arcpy.GetCount_management(shapefile_layer_name)}") # if selected 1 it will return 1

current_time = datetime.datetime.now()
print("Current Time:", current_time)

Count of shape of c1823_ADRSM_01ha : 1206
Current Time: 2024-07-25 17:33:02.212162


#### Basic object fetched

In [46]:
# Get Project, layout, map, mapview, mapframe
# Before using this code , you should open and load both layout window and Map window, or it will erro: mapview is None

current_project=arcpy.mp.ArcGISProject("CURRENT")
layout=current_project.listLayouts()[1]  # 0 is the dataframe drive layout(text,area) # 1 is the 
amap=current_project.listMaps()[0]  
mv=current_project.activeView    #mv=amap.defaultView
mf= layout.listElements("MAPFRAME_ELEMENT")[0]  #get the mapframe in the layout

print("Current layout name:",layout.name)
print("Mapview.extent: ", mv.camera.getExtent())
print("Mapname: ",amap.name)
print("Mfname: ",mf.name)
current_time = datetime.datetime.now()
print("Current Time:", current_time)

Current layout name: JpegExport
Mapview.extent:  -8528335.1379518 -1274334.40317018 -8527337.73216469 -1272991.85849027 NaN NaN NaN NaN
Mapname:  Map
Mfname:  Map Frame
Current Time: 2024-07-25 17:33:08.271362


## Exporting func

#### Get Imagery date function

In [47]:
import requests

def get_imagery_date(lat, lon):
    # URL for the ESRI World Imagery REST endpoint
    url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/identify"
    
    # Parameters for the identify operation
    params = {
        'geometry': f'{lon},{lat}',
        'geometryType': 'esriGeometryPoint',
        'sr': 4326,
        'layers': 'all',
        'tolerance': 2,
        'mapExtent': f'{lon-0.01},{lat-0.01},{lon+0.01},{lat+0.01}',
        'imageDisplay': '800,600,96',
        'returnGeometry': False,
        'f': 'json'
    }
    
    # Make the request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract and print relevant metadata
    if 'results' in data and len(data['results']) > 0:
        for result in data['results']:
            if 'attributes' in result:
                attributes = result['attributes']
                # Extract specific metadata fields (e.g., acquisition date, source, etc.)
                if 'AcquisitionDate' in attributes:
                    print(f"Acquisition Date: {attributes['AcquisitionDate']}")
                if 'Source' in attributes:
                    print(f"Source: {attributes['Source']}")
                if 'DATE (YYYYMMDD)' in attributes:
                    if attributes['DATE (YYYYMMDD)'] is not None:
                        return attributes['DATE (YYYYMMDD)']
                    print(f"Date: {attributes['DATE (YYYYMMDD)']}")
                # Print all available attributes for inspection
                
    else:
        print("No imagery information found for the specified location.")


def get_lat_long(camera):
    # Get the center of the map view
    # Calculate the center point of the extent
    extent = camera.getExtent()
    center_x = (extent.XMin + extent.XMax) / 2
    center_y = (extent.YMin + extent.YMax) / 2
    
    # Create a point geometry for the center point
    center_point = arcpy.PointGeometry(arcpy.Point(center_x, center_y), extent.spatialReference)
    
    # Define the target spatial reference (WGS84)
    target_sr = arcpy.SpatialReference(4326)
    
    # Project the center point to WGS84
    center_point_wgs84 = center_point.projectAs(target_sr)
    
    # Extract the coordinates in decimal degrees
    latitude = center_point_wgs84.centroid.Y
    longitude = center_point_wgs84.centroid.X
    return latitude, longitude

#### Export function

In [48]:
# this function is for export the images 
# it can be call onetime or iterate all in different year and month datasource set
import os

#only based on the shape "1823_ADRSM" and the 
# default parameter
fc = shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]
default_path=r"C:\Users\lycaz\Desktop\output"
file_type='jpg'
min_area=min_area

def export_image(path=default_path,src_label="EsriWorldImagery",dpi=150,f_type=file_type,istest=False):
    """
    Brief description of the function.

    Parameters:

    time (str): Description of param2.
    path(str): which folder you want to saved
    
    Returns:
    type: Description of what the function returns.
    """
    print(f"config:, path:{path}, label:{src_label}, dpi={dpi},f_type{f_type}")
    
    if os.path.exists(path)==False:
        print("Path does not exist.")
        return
    
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        count_im=0
        for row in cursor:
            #print(u'{0}, {1},{2}'.format(row[2],row[0], row[1]))
            
            print(f" Processing index {row[2]} {row[0]}", end="\r")
            if row[1] > min_area: 
                count_im+=1
                print(f"image coount {count_im}", end="\r")
                geometry=row[3]
                mv.camera.setExtent(geometry.extent)
                amap.defaultCamera=mv.camera
                amap.defaultCamera.scale*=2
                mf.camera.setExtent(amap.defaultCamera.getExtent())
                lati,longi=get_lat_long(mf.camera)
                date_stamp=get_imagery_date(lati,longi)
                print(f"Current Lat:{lati}, Long:{longi}, Date:{date_stamp}")
                if row[1]<4:
                    desired_scale=5/row[1]
                    mf.camera.scale*=desired_scale
                else:
                    mf.camera.scale*=2
                ele=layout.listElements(wildcard="Text")[0]
                ele.text=f"{row[2]}-{date_stamp}-{row[1]:.2f}ha"
                ele_source=layout.listElements(wildcard="Text 1")[0]
                ele_source.text=src_label
                # if f_type=='jpg':
                #     layout.exportToJPEG(f"{path}\\{str(row[2])}-{date_stamp}-{src_label}.jpg",resolution=dpi)
                # elif f_type=="tif":
                #     layout.exportToTIFF(f"{path}\\{str(row[2])}-{date_stamp}-{src_label}.tif") # without geotif

            if istest==True:
                break
                    
                
        print('\n')
        print(f"Complete current the image in: {time} ")
    
    
print(f"Exporting function ready path:{default_path}")   
    
current_time = datetime.datetime.now()
print("Current Time:", current_time)

Exporting function ready path:C:\Users\lycaz\Desktop\output
Current Time: 2024-07-25 17:33:17.363557


## Excute export all the shapes' images

In [ ]:

# Before exporting, confirm the layout is setting well, the tile map is loaded and the shapefile is outlined.


# get the certain shape of shapefile  and iterate the shapefile
# handle the mv (mapview), do not effect the layout 
import time 
start_time = time.time()

fc = shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]

path=r"C:\Users\lycaz\Desktop\output"    # schange it before exporting !!!!!!
datasource_str="worldimagery" 

# call the exporting function for all the shapes in the shapefile
export_image(path,datasource_str,150,"jpg",False)

stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")
current_time = datetime.datetime.now()
print("Current Time:", current_time)

config:, path:C:\Users\lycaz\Desktop\output, label:worldimagery, dpi=150,f_typejpg
Lat:-12.135973026688887, Long:-75.24903611262647
Lat:-6.2485940006078735, Long:-79.66107437972877ritos
Lat:-7.807486331512859, Long:-77.64293549737525
Lat:-12.8345802929423, Long:-76.02301579514277ga
Lat:-6.725945615987959, Long:-76.25526990393287
Lat:-10.00543356495644, Long:-77.13246042450893
Lat:-13.58234304386884, Long:-71.41688854561986
Lat:-17.07857656100099, Long:-70.86770450835633a
Lat:-12.642937348328381, Long:-73.07202255649574
Lat:-15.092733476911372, Long:-69.8173815800958achi-C.C. Taurahuta
Lat:-12.751728369446727, Long:-74.12661839492296
Lat:-1.7893522049580077, Long:-73.40783559133942d Santa Rosa-Rosa Panduro
Lat:-13.2021648904659, Long:-70.68698732825257
Lat:-14.329629897187182, Long:-70.07554705550217
Lat:-10.236259903781663, Long:-77.08311380500805
Lat:-8.877072774302585, Long:-76.92662575882166
Lat:-14.139729308882947, Long:-70.39162609491831
Lat:-7.704529707070637, Long:-77.7020579458

Lat:-9.080751807329126, Long:-77.09272387405754ajra
Lat:-9.065294913322434, Long:-77.08404177315929
Lat:-9.013644434146407, Long:-77.46398522698658cro
Lat:-8.972066131346555, Long:-77.80290637053253nta
Lat:-8.948294800159278, Long:-77.84649632079852
Lat:-8.957469682718118, Long:-77.36495662629721Pampa
Lat:-8.934961830480182, Long:-77.31173021457768ga
Lat:-8.827858157914223, Long:-77.92014221549081hpunru
Lat:-15.538077769491903, Long:-74.83958953662778
Lat:-13.428074618390587, Long:-70.37410799843536
Lat:-7.7023314491095975, Long:-77.74334251909197
Lat:-7.143624308370303, Long:-77.85888899747921
Lat:-8.473196080636754, Long:-77.29369155497123
Lat:-8.41687240453888, Long:-77.31354304637445
Lat:-11.723266554441782, Long:-75.76822673937251naco
Lat:-5.283748804328184, Long:-79.46558401883736iento de Residuos Solidos
Lat:-5.081928915767725, Long:-80.17965833416473
Lat:-4.887495967302649, Long:-80.62870386074454
Lat:-5.403690251983254, Long:-80.68121863308343
Lat:-5.3228150639317935, Long:-80

Lat:-11.369493507382817, Long:-75.559147290515
Lat:-17.203121034883978, Long:-70.90174499713389io N°01
Lat:-3.6758865141932833, Long:-80.2289812003221 El Algarrobal
Lat:-7.2898437468149195, Long:-76.76686683923322
Lat:-10.770169543029585, Long:-75.78541854787767
Lat:-10.736861849516167, Long:-73.78559025808389al
Lat:-6.655933778590987, Long:-76.30529407041497
Lat:-14.902895520523, Long:-70.86159996381723
Lat:-14.882128626032555, Long:-69.73581523788289
Lat:-8.611428376370986, Long:-74.96241191168032
Lat:-8.76558248250634, Long:-77.18396575281719Capilla
Lat:-11.794072907787148, Long:-75.46437334690242
Lat:-13.44050145033793, Long:-72.14697413929505
Lat:-7.2393601715838605, Long:-74.96233911246003la Disposición Final de Residuos Sólido Municipales
Lat:-8.63295733580987, Long:-74.31381633565533
Lat:-9.302654487400163, Long:-74.47871702966714
Lat:-8.826668283162254, Long:-75.03807310389058
Lat:-7.882185785511125, Long:-76.6744211239357
Lat:-7.068066160229963, Long:-76.70911029010779or Situ

### Test the export function

In [36]:
import time
import arcpy

stat_time = time.time()

fc=shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]
path=r"C:\Users\lycaz\Desktop\output"    
data_source="wordimagery" 
time_stamp="202403"
export_image(time_stamp,path,data_source,150,"jpg",True)


end_time = time.time()
elapsed_time = end_time - stat_time

print(f"Start time: {time.ctime(stat_time)}")
print(f"Stop time: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

config:, path:C:\Users\lycaz\Desktop\output, label:wordimagery, dpi=150,f_typejpg
image coount 1dex 50 Botadero Purupampa

Complete current the image in: 202403 
Start time: Thu Jul 25 16:48:40 2024
Stop time: Thu Jul 25 16:48:43 2024
Elapsed time: 2.21 seconds


#### Test the get_date function


In [44]:

# Example usage
latitude = -10.09337942  # Replace with your latitude
longitude = -76.6385212  # Replace with your longitude
print("date:",get_imagery_date(latitude, longitude))

import arcpy

# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Get the active map view
map_view = aprx.activeView

# Check if there's an active view
if map_view:
    # Get the camera object
    camera = map_view.camera
    
    # Get the camera's extent
    extent = camera.getExtent()
    
    # Calculate the center point of the extent
    center_x = (extent.XMin + extent.XMax) / 2
    center_y = (extent.YMin + extent.YMax) / 2
    
    # Create a point geometry for the center point
    center_point = arcpy.PointGeometry(arcpy.Point(center_x, center_y), extent.spatialReference)
    
    # Define the target spatial reference (WGS84)
    target_sr = arcpy.SpatialReference(4326)
    
    # Project the center point to WGS84
    center_point_wgs84 = center_point.projectAs(target_sr)
    
    # Extract the coordinates in decimal degrees
    latitude = center_point_wgs84.centroid.Y
    longitude = center_point_wgs84.centroid.X
    print("date:",get_imagery_date(latitude, longitude))
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("No active map view found.")

date: 20210809
date: 20220816
Latitude: -11.366217971175205, Longitude: -76.60685810084154


## Exporting image by time series

### For nicfi 
The code is for nicfi which is load by google earth engine.

Using iteration load multiple layer then do the exporting function on each layer

In [22]:
# Function for exporting history images in 5x12 month from nicfi source , n shapes x 60 images
# Get 60 tile layers and add it to imageview each layer export n shape. There is no built in function to change one layer and update the layer to certain date.


# Check the times. 
# from "2017-01-01" to "2024-05-01"  to get 50 date from the collection
start_date="2017-01-01"
end_date="2024-4-01"

nicfi_collection=nicfi.filter(ee.Filter.date(start_date,end_date)).sort('system:time_start', False)
dates=nicfi_collection.aggregate_array('system:time_start')
# Convert dates to a list and label_list
dates_list = dates.getInfo()
date_lables=[]
# Iterate over the dates and print them out
print(len(dates_list))
for date in dates_list:
    date_lables.append(ee.Date(date).format("YYYY-MM-dd").getInfo())
print("Date:",date_lables)

current_time = datetime.datetime.now()
print("Current Time:", current_time)

50
Date: ['2024-03-01', '2024-02-01', '2024-01-01', '2023-12-01', '2023-11-01', '2023-10-01', '2023-09-01', '2023-08-01', '2023-07-01', '2023-06-01', '2023-05-01', '2023-04-01', '2023-03-01', '2023-02-01', '2023-01-01', '2022-12-01', '2022-11-01', '2022-10-01', '2022-09-01', '2022-08-01', '2022-07-01', '2022-06-01', '2022-05-01', '2022-04-01', '2022-03-01', '2022-02-01', '2022-01-01', '2021-12-01', '2021-11-01', '2021-10-01', '2021-09-01', '2021-08-01', '2021-07-01', '2021-06-01', '2021-05-01', '2021-04-01', '2021-03-01', '2021-02-01', '2021-01-01', '2020-12-01', '2020-11-01', '2020-10-01', '2020-09-01', '2020-06-01', '2019-12-01', '2019-06-01', '2018-12-01', '2018-06-01', '2017-12-01', '2017-06-01']
Current Time: 2024-04-26 17:02:22.033660


In [26]:
# iterate the time and export in each iteration
# this block of code is independency from the 

# find the shapefile layer and always put it to top

import time 
start_time = time.time()

target_layer_name="1823_ADRSM"
top_layer=None
for lyr in amap.listLayers():
    if lyr.name==target_layer_name:
        top_layer=lyr

def put_layer_top(ta_map=amap,ta_layer=top_layer):
    if ta_layer is not None:
        ta_map.moveLayer(amap.listLayers()[0],ta_layer,"BEFORE")
        print(f"{ta_layer}Layer moved to the top successfully.")
    else:
        print("Layer not found in the map.")

def turn_off_other_layer():
    for ly in amap.listLayers():
        if ly.name!=target_layer_name:
            ly.visible = False


export_path=r"C:\Users\lycaz\Desktop\output"
label_text="nicfi"
vis2= {'bands':['R','G','B'],'min':300,'max':3500,'gamma':1.5}
vis_n={'bands':['N'],'min':1000,'max':8000,'gamma':2.0}

for i,date in enumerate(dates_list):
    month=ee.Date(date).get('month').getInfo()
    s_d=ee.Date(date).format("YYYY-MM-dd").getInfo()
    e_d=ee.Date(date).advance(1, 'month').format("YYYY-MM-dd").getInfo()
    print("Adding Date Range:",s_d,e_d)
    image=nicfi.filter(ee.Filter.date(s_d,e_d)).sort('system:time_start', False).first()
    layer_name = ee.Date(date).format("YYYY-MM-dd").getInfo()
    turn_off_other_layer() # turn off all layer
    m.addLayer(image,vis2,layer_name) # add new layer   # change here for rgb or n 
    put_layer_top(amap,top_layer) # # put the layer top
    
    # print export image start time 
    sta_time = time.time()
    print(f"Start {layer_name} exportin time: {time.ctime(sta_time)}")
    export_image(layer_name.replace("-","")[:6],True,export_path,label_text,150,'tif') #150dpi would save time for 38file/min
    sto_time = time.time()
    print(f"End {layer_name} exportin time: {time.ctime(sto_time)}")
    
    
stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds") 


Adding Date Range: 2024-03-01 2024-04-01
1823_ADRSMLayer moved to the top successfully.
Start 2024-03-01 exportin time: Fri Apr 26 17:04:37 2024
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "projects/planet-nicfi/assets/basemaps/americas"
      }
    }
  }
}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202403 
End 2024-03-01 exportin time: Fri Apr 26 17:05:56 2024
Adding Date Range: 2024-02-01 2024-03-01
1823_ADRSMLayer moved to the top successfully.
Start 2024-02-01 exportin time: Fri Apr 26 17:05:57 2024
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load"

}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202204 
End 2022-04-01 exportin time: Fri Apr 26 17:32:14 2024
Adding Date Range: 2022-03-01 2022-04-01
1823_ADRSMLayer moved to the top successfully.
Start 2022-03-01 exportin time: Fri Apr 26 17:32:15 2024
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "projects/planet-nicfi/assets/basemaps/americas"
      }
    }
  }
}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202203 
End 2022-03-01 exportin time: Fri Apr 26 17:33:24 2024
Adding Date Range: 2022-02-01 2022-0

}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202104 
End 2021-04-01 exportin time: Fri Apr 26 17:46:59 2024
Adding Date Range: 2021-03-01 2021-04-01
1823_ADRSMLayer moved to the top successfully.
Start 2021-03-01 exportin time: Fri Apr 26 17:47:00 2024
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "projects/planet-nicfi/assets/basemaps/americas"
      }
    }
  }
}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202103 
End 2021-03-01 exportin time: Fri Apr 26 17:48:15 2024
Adding Date Range: 2021-02-01 2021-0

}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 201806 
End 2018-06-01 exportin time: Fri Apr 26 18:03:29 2024
Adding Date Range: 2017-12-01 2018-01-01
1823_ADRSMLayer moved to the top successfully.
Start 2017-12-01 exportin time: Fri Apr 26 18:03:30 2024
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "projects/planet-nicfi/assets/basemaps/americas"
      }
    }
  }
}), dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 201712 
End 2017-12-01 exportin time: Fri Apr 26 18:04:56 2024
Adding Date Range: 2017-06-01 2017-0

### For sentinel
Sentinel2 data is from LivingAtlas. which can be control by  the arcgis time slide

Sentinel2 can be load from several different source (livingatlas,  Sentinel-2 on AWS collections [ArcGISserver](https://sentinel.arcgis.com/arcgis/rest/services/Sentinel2/ImageServer)...)
LivingGatlas is best performance

In [13]:
# Setting the config before export
# several parameters
senti_layer_name="Sentinel2L2A"
senti_path=r"C:\Users\lycaz\Desktop\senti"
senti_file_type="jpg"
senti_label_text="sentinel2"

current_time = datetime.datetime.now()
print("Current Time:", current_time)

Current Time: 2024-04-28 20:32:44.966307


In [14]:
# check the currently the update of the sentinel2 layer's date :
# the result acturrly is set in the time properties in the arcgis. 
sentinel_layer=None
# get the sentinel layer
for lyr in amap.listLayers():
    if lyr.supports('TIME'):
        if lyr.name==senti_layer_name:
            lyr.isTimeEnabled==True
            # get the sentinel2l2a layer with time series
            print(lyr.time.startTime, lyr.time.endTime)  # check the currently time of layer
current_time = datetime.datetime.now()
print("Current Time:", current_time)

2015-07-04 10:10:06 2024-04-17 14:37:49
Current Time: 2024-04-28 20:32:52.008821


In [15]:
# each month export one image for one certain shape also collect the 5x12 60 timeseriel image for each shape
import arcpy, os
import datetime
from dateutil.relativedelta import relativedelta
import time 
start_time = time.time()

# using the maptime to change whole map's time
t= mf.time
# set the timeend before time end

# export the image based on the time 
t.currentTimeEnd=datetime.datetime(2024,4,1,0,0)
t.currentTimeStart=datetime.datetime(2024,3,1,0,0)
t.setTimeInclusion('INCLUDE_ONLY_START')
for i in range(0,60):
    print("Exporting time index:",i+1)
    # excut the export
    print(str(t.currentTimeStart).replace("-","")[:6])
    export_image(str(t.currentTimeStart).replace("-","")[:6],True,senti_path,senti_label_text,150,senti_file_type)
    print("Loaded: ",t.currentTimeStart ,t.currentTimeEnd)
    t.currentTimeEnd=t.timeStep(t.currentTimeEnd,-1,"months")
    t.currentTimeStart=t.timeStep(t.currentTimeStart,-1,"months")

stop_time= time.time()
elapsed_time = stop_time - start_time
print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")    
     


Exporting time index: 1
202403
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:nicfi, dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202403 
Loaded:  2024-03-01 00:00:00 2024-04-01 00:00:00
Exporting time index: 2
202402
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:nicfi, dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202402 
Loaded:  2024-02-01 00:00:00 2024-03-01 00:00:00
Exporting time index: 3
202401
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:nicfi, dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete cu

 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202006 
Loaded:  2020-06-01 00:00:00 2020-07-01 00:00:00
Exporting time index: 47
202005
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:nicfi, dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202005 
Loaded:  2020-05-01 00:00:00 2020-06-01 00:00:00
Exporting time index: 48
202004
config: isScale:True, path:C:\Users\lycaz\Desktop\output, label:nicfi, dpi=150,f_typejpg
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202004 
Loaded:  2020-04-01 00:00:00 2020-05-01 00:00:00
Exporting time index: 49
202003
config: i

NameError: name 'elapsed_time' is not defined

## Other function

### Center certain shape

In [12]:
# [] TODO: using gee load sentinel faild ,can not show the map

senti=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate('2020-01-01', '2024-01-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
visualization = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2']
}
m.addLayer(senti.mean(), visualization, 'senti');

In [ ]:
# mapTime using  layertime using test
t= mf.time
# set the timeend before time end
t.currentTimeEnd = datetime(2020,1,1,0,0)
t.currentTimeStart = datetime(2019,1,1,0,0)

t.setTimeInclusion('INCLUDE_ONLY_START')

print(mf.time.currentTimeStart, mf.time.currentTimeEnd)






In [31]:
# Function for timely review the imagry
# iterate all time from recently 5 years 
# show it in certain place 



# function to query certain shape return the geometrys , using index ,using size ha as query_str
def query_geometry(query_str):
    geometries = []
    # Use a search cursor to iterate over the features that match the query
    with arcpy.da.SearchCursor("1823_ADRSM", ['AREA_HA','SHAPE@'], where_clause=query_str) as cursor:
        for row in cursor:
            # Access the geometry of each feature
            geometry = row[1]
            # Add the geometry to the list
            geometries.append(geometry)
            print(row[0])
            print(type(row[1]))
    return geometries


# showcase of using query_geometry and center it 
sql_query="Index=1171"
selected_geo=query_geometry(sql_query)
if len(selected_geo)==1:
    print(selected_geo)
    print(selected_geo[0].extent.height,selected_geo[0].extent.width)
    # center to this geo
    #mv.panToExtent(selected_geo[0].extent)
    #mv.camera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.scale*=1.5
    
    print(mv.camera.getExtent ())

    

    

181.24151696
<class 'arcpy.arcobjects.geometries.Polygon'>
[<Polygon object at 0x1f38ab44070[0x1f30ed3fde0]>]
0.025418119999999433 0.019702124999994908
-76.116801368 -13.3816299269999 -76.097099243 -13.3562118069999 0 0 nan nan


In [10]:
# TODO 
# change basemap
start_year=2019
end_year=2024
date_list=[]
for i in range(start_year,end_year):
    for j in range(3,13):
        if j >9:
            date_list.append(f"{i}-{j}-01")
        else:
            date_list.append(f"{i}-0{j}-01")
    date_list.append(f"{i+1}-01-01")
    date_list.append(f"{i+1}-02-01")
print(len(date_list),date_list)
 

60 ['2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']


In [82]:
# Get the url of the tile layer from GEE.
from geemap.ee_tile_layers import _get_tile_url_format

basemap=nicfi.filter(ee.Filter.date('2019-01-01','2023-03-11')).sort('system:time_start', False).first()

print(type(basemap))
# Define visualization parameters
vis={'bands':['R','G','B'],'min':500,'max':3500,'gamma':2.0}

# Get the tile URL format
map_id_dict = ee.Image(basemap).getMapId(vis)
print(map_id_dict)
url = map_id_dict["tile_fetcher"].url_format
print(url)

url_format = _get_tile_url_format(basemap, vis)
print(url_format)

<class 'ee.image.Image'>
{'mapid': 'projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132', 'token': '', 'tile_fetcher': <ee.data.TileFetcher object at 0x000001F39CC21070>, 'image': <ee.image.Image object at 0x000001F39CC4F4F0>}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132/tiles/{z}/{x}/{y}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-e42174e0132e6abec57d4116f009c856/tiles/{z}/{x}/{y}
